# Filtering Junk Listings

Author: Nathan Robertson

Based on work done in earlier MADS project milestone courses, I know that sometimes there are junk listings on home buying and renting platforms. Junk listings are ones that look suspicious - as in, it's unclear if the home is being represented honestly, or if the lister has a right to sell the home. These listings will throw off the model's performance, because they likely have metadata that is not correct and will create more noise for the model.

This workbook will use `gpt-4o-mini` to classify listings as either junk or not -- and will provide a reason why if it thinks it is junk. The output will be a data set of zillow IDs and their ruling of junk or no junk, which will then be used to filter out the junk listings in the final model.

In [1]:
# AI for autogenerating image labels.
import openai
client = openai.OpenAI(api_key='')

# Data manipulation
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# For tracking iterations
from tqdm import tqdm

In [2]:
# Load data. Need to do a little finagling to get the pre-processed, original
# description data out of the old cleaned zillow file.

df = pd.read_csv('data/backup/BACKUP cleaned_zillow_data.csv')
original = pd.read_csv('data/backup/BACKUP zillow_listing_data.csv')

original = original[['zillowId','description']]
original.columns = ['zillowId','description_original']

df = pd.merge(df, original, how='left', on='zillowId')

# We no longer need this.
del original

/var/folders/0l/38yb605j6z30lbrmlsfhrq0h0000gn/T/ipykernel_38524/2628749678.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  original = pd.read_csv('data/backup/BACKUP zillow_listing_data.csv')


In [3]:
df.head()

,Unnamed: 0,zillowId,homeStatus,streetAdress,city,state,zipcode,price,bedrooms,bathrooms,yearBuilt,zestimate,homeType,livingArea,hasGarage,parkingCapacity,sewer,hasHeating,fireplaces,hasWaterfrontView,isSeniorCommunity,levels,lotSize,hoaFee,buyersFeePerc,daysOnZillow,views,favorites,latitude,longitude,mainPhoto,brokerage,percSellFaster,propertyTax,mortgageRate,listingUserId,agentName,agentRating,agentRecentSales,agentReviewsCount,photosList,isForeclosure,isBankOwned,rentZestimate,totalActualRent,annual_house_insurance,price_history,description,similarHomes,termsCash,termsConventional,termsFHA,termsVA,schoolAverageDistance,schoolContainsCharter,schoolRatingElementary,schoolRatingMiddleSchool,schoolRatingHighSchool,hasPatioPorch,ngram_dolean,ngram_options vast endless,ngram_possibilities,ngram_community design exclusive,ngram_estate,ngram_stewarded,ngram_lucite,ngram_arclinea,ngram_marble block kitchen,ngram_downtown ocean best,ngram_brand arclinea entertainers,ngram_bathrooms magnificient guest,ngram_macassar,ngram_wine,ngram_masterpiece,ngram_private,ngram_cellar,ngram_infinity,ngram_expansive,ngram_intracoastalview,ngram_designed,ngram_chefs,ngram_parties grand,ngram_court tennis house,ngram_regencystyle,ngram_art deco gates,ngram_imposing main house,ngram_mansions dotting nearby,ngram_luxury,ngram_unparalleled,ngram_marble,ngram_architect,ngram_theater,ngram_outside scenery,ngram_staff,ngram_massage room,ngram_custom,ngram_magnificent,ngram_extraordinary,ngram_privately owned island,ngram_gourmet,ngram_iconic,ngram_stunning,ngram_infinityedge,ngram_panoramic,ngram_unobstructued sunrise sunset,ngram_aged white,ngram_experience perfect relaxation,ngram_indooroutdoor,ngram_court,ngram_architectural,ngram_subzero,ngram_rarefied,ngram_antiquities,ngram_sauna,ngram_terrace,ngram_saota,ngram_italian,description_original
0,2,23352313.0,FOR_SALE,7001 Palm Lane,Anacortes,WA,98221.0,2498000.0,4.0,3.0,2000.0,NaN,SingleFamily,3788.0,True,4.0,sewer,True,2.0,False,False,1.0,10.300000,0.0,2.5,125.0,1219.0,40.0,48.441994,-122.591225,https://photos.zillowstatic.com/fp/8a6b6abe798...,COMPASS,0.25,0.83,6.606,X1-ZUyt9ezy9pvtop_61q4w,Eileen Hebert,5.0,1.0,259.0,"[""https://photos.zillowstatic.com/fp/8a6b6abe7...",False,False,NaN,0.0,10491.6,"[[""2023-10-11"", 2498000], [""1994-06-27"", 308000]]",exquisite equestrian oasis perfect blend luxur...,"[84763318, 23352179, 23377656, 23387845, 20536...",False,True,False,False,3.200000,False,9.0,5.0,9.0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,Exquisite Equestrian Oasis. The perfect blend ...
1,3,23380009.0,FOR_SALE,14820 Gibralter Road,Anacortes,WA,98221.0,5300000.0,5.0,6.0,2001.0,NaN,SingleFamily,5577.0,True,4.0,sewer,True,4.0,False,False,1.0,17.010000,0.0,2.5,276.0,2146.0,127.0,48.429980,-122.587870,https://photos.zillowstatic.com/fp/3a101f623ea...,"Windermere R.E. Northeast, Inc",0.22,0.83,6.606,X1-ZUz3sd58bhvnd5_3qul1,bryanloveless5250,5.0,0.0,9.0,"[""https://photos.zillowstatic.com/fp/3a101f623...",False,False,NaN,0.0,22260.0,"[[""2023-03-21"", 5300000], [""2015-11-30"", 21500...",saratoga stablesa magnificent gated acre eques...,"[2055432779, 2053658449, 228491857, 82573373, ...",False,True,False,False,4.033333,False,9.0,5.0,9.0,False,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Saratoga Stables–a magnificent gated 17+ acre ...
2,4,23352179.0,FOR_SALE,7270 Old Brook Lane,Anacortes,WA,98221.0,1499000.0,5.0,4.0,1983.0,1391500.0,SingleFamily,2968.0,True,2.0,sewer,True,2.0,False,False,2.0,9.010000,0.0,2.5,94.0,936.0,29.0,48.460000,-122.587746,https://photos.zillowstatic.com/fp/99488e836da...,Windermere RE Anacortes Prop.,0.30,0.83,6.606,X1-ZUz4p08auiqhhl_3wgux,Colleen Craig,5.0,0.0,9.0,"[""https://photos.zillowstatic.com/fp/99488e836...",False,False,5818.0,0.0,6295.8,"[[""2023-09-19"", 1499000], [""2013-07-03"", 1900]...",lovely cape code style home nor

In [33]:
def prepListingData(row):

    data = {
        'price': row['price'],
        'homeType': row['homeType'],
        'bedrooms': row['bedrooms'],
        'bathrooms': row['bathrooms'],
        'livingArea': row['livingArea'],
        'description': row['description_original']
    }

    return data

In [37]:
# Define a few helper functions that will be used.
def prompt(listingData):
    return [
        {'role': 'system', 
         'content': f"""You are a robot that detects spam or junk listings. You will compare the description 
         of the home to its metadata, as well as assess the description itself. If you do not detect anything 
         suspicious, you MUST return the boolean False. If you detect anything HIGHLY suspicious that makes you think the 
         listing is not real or that the lister does not have a right to sell the home, you MUST return a 
         tuple including the boolean True in the first position, and the rationale for the decision in the second 
         position. If you are unsure, you MUST return False. Junk / spam listings are ones where the description 
         suggests the listing is suspicious in some way. Highly embellished language describing the listing does NOT 
         classify as junk / spam listings. ONLY return either True or False. You have a high bar for declaring
         a listing "Junk" -- only return True is your are very confident this looks like a spam or junk listing."""},
        {'role': 'user', 
         'content': f"""True or False: Is this listing a spam or junk listing? Listing data: {listingData}"""}
            ]

def chatgpt(message,model='gpt-4o-mini'):
        response = client.chat.completions.create(
                      model=model,
                      messages=message)
            
        content = response.choices[0].message.content
        return content

In [ ]:
isJunkListingResults = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Checking for spam/junk listings..."):

    listingData = prepListingData(row)
    myPrompt = prompt(listingData)
    isJunkListing = chatgpt(message=myPrompt)
    isJunkListingResults.append(isJunkListing)

Checking for spam/junk listings...:   3%|▏       | 3799/129247 [34:46<18:13:02,  1.91it/s]